In [1]:
# core python libraries we rely on
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr

# SimPEG ecosystem 
import discretize
from discretize.utils import mkvc
from SimPEG.electromagnetics import frequency_domain as fdem
from pymatsolver import Pardiso
import ipywidgets
%load_ext memory_profiler

In [2]:
data = np.load('../marlim_comp.npz')  #  or 'marlim_orig.npz'
tres_h = data['res_h']
tres_v = data['res_v']

# We have to add an air layer
hz = np.r_[data['hz'], 20]

mesh_full = discretize.TensorMesh(
    [data['hx'], data['hy'], hz], x0=data['x0'])

# Add air resistivity
res_h_c = 1e8*np.ones(mesh_c.vnC)
res_v_c = 1e8*np.ones(mesh_c.vnC)
res_h_c[:, :, :-1] = tres_h
res_v_c[:, :, :-1] = tres_v
del data, tres_h, tres_v

mesh_full

FileNotFoundError: [Errno 2] No such file or directory: '../marlim_comp.npz'